In [701]:
homes_df = pd.read_csv("desktop/City_Zhvi_AllHomes.csv")

university_df = pd.read_csv("desktop/university_towns.txt", delimiter = "edit", header = None, names = ["a","b"], engine="python")
university_df2 = pd.DataFrame(columns={"State", "RegionName"})
state = ""
region = ""
for i in university_df["a"]:
    if i[-1] == "[":
        state = i[:-1]
    elif i[-1] == "]" or i[-1] == ")":
        region = i.split("(")[0]
        university_df2 = university_df2.append(pd.DataFrame([[state, region]], columns = ["State", "RegionName"]), sort=True)   

        
gdp_df = pd.read_excel("desktop/gdplev (1).xls",skiprows=5)
gdp_df = gdp_df[214:].drop(columns={"Unnamed: 0","GDP in billions of current dollars","GDP in billions of chained 2009 dollars","Unnamed: 3","Unnamed: 7","GDP in billions of current dollars.1"})
gdp_df.rename(columns={"Unnamed: 4":"Quarters"}, inplace = True)
gdp_df.columns = ["Quarters","GDP"]


university_df2["RegionName"] = university_df2["RegionName"].str.strip()


In [700]:
from scipy.stats import ttest_ind

gdplev = pd.ExcelFile("desktop/gdplev (1).xls")
gdplev = gdplev.parse("Sheet1", skiprows = 219)
gdplev = gdplev[["1999q4", 9926.1]]
gdplev.columns = ["Quarters","GDP"]
def recessionStart():
    for i in range(2,len(gdplev)):
        if gdplev.iloc[i-2][1]>gdplev.iloc[i-1][1] and gdplev.iloc[i-1][1] > gdplev.iloc[i][1]:
            return gdplev.iloc[i-2]
        
def recessionEnd():
    start = recessionStart()
    gdplev_2 = gdplev[start.name:]
    for i in range(2, len(gdplev_2)):
        if gdplev_2.iloc[i-2][1]<gdplev_2.iloc[i-1][1] and gdplev_2.iloc[i-1][1] <gdplev_2.iloc[i][1]:
            return gdplev_2.iloc[i]
        
def getRecessionBottom():
    start = recessionStart()
    end = recessionEnd()
    gdplev_2 = gdplev[start.name:end.name+1]
    return gdplev_2[gdplev_2["GDP"] == gdplev_2["GDP"].min()]      

df_homes = homes_df

def convertQuarters():
    new_df = pd.DataFrame(columns=["RegionName","State"])
    new_df[["RegionName","State"]] = new_df[["RegionName","State"]].append(df_homes[["RegionName","State"]])
 
    for i in homes_df.columns:
        if i[-2:] == "01":
            new_df[i[:4] + "q1"] = df_homes[i]
        elif i[-2:] in ["02","03"]:
            new_df[i[:4] + "q1"] += df_homes[i]
        
        if i[-2:] == "04":
            new_df[i[:4] + "q2"] = df_homes[i]
        elif i[-2:] in ["05","06"]:
            new_df[i[:4] + "q2"] += df_homes[i]
            
        if i[-2:] == "07":
            new_df[i[:4] + "q3"] = df_homes[i]
        elif i[-2:] in ["08","09"]:
            new_df[i[:4] + "q3"] += df_homes[i]
 
        if i[-2:] == "10":
            new_df[i[:4] + "q4"] = df_homes[i]
        elif i[-2:] in ["11","12"]:
            new_df[i[:4] + "q4"] += df_homes[i]            
        else:
            continue
            
    return new_df

housing_data = convertQuarters()
df1 = housing_data.loc[:,"2000q1":]
df2 = housing_data.loc[:,:"State"]

housing_result = pd.concat([df2, df1], axis=1)
recStart = recessionStart()
recEnd = recessionEnd()
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

housing_result["States"] = [states[i] for i in housing_result["State"]]
university_list = university_df2["RegionName"].tolist()


housing_result["isUni"] = [1 if housing_result["RegionName"][i] in university_list else 0 for i in range(len(housing_result["RegionName"]))] 
housing_result.set_index(["RegionName","State"], inplace= True)


housing_result = pd.concat([housing_result.loc[:,recStart[0]:recEnd[0]], housing_result["isUni"]], axis=1)
#housing_result["Ratio"] = housing_result[recStart[0]]/housing_result.min(axis=0)
housing_result["Score"] = abs(housing_result["2008q3"] - housing_result.loc[:,"2008q3":"2009q4"].min(axis=1))+ abs(housing_result.loc[:,"2008q3":"2009q4"].min(axis=1) - housing_result["2009q4"])


is_uni = housing_result[housing_result["isUni"]==1]["Score"].dropna()
not_uni = housing_result[housing_result["isUni"]==0]["Score"].dropna()

pval = ttest_ind(is_uni, not_uni)[1]


8.557239568748945e-09